In [ ]:
import numpy as np

class AdvancedKalmanFilter:
    def __init__(self, initial_detection, process_noise_std=0.1, measurement_noise_std=1.0):
        # Initialize state and covariance matrices
        self.x = np.array([[float(initial_detection.x)], [float(initial_detection.y)], [float(initial_detection.vx)], [float(initial_detection.vy)]])
        self.P = np.eye(4) * 500.0  # High initial uncertainty
        self.F = np.array([[1.0, 0.0, 1.0, 0.0],  # State transition model
                           [0.0, 1.0, 0.0, 1.0],
                           [0.0, 0.0, 1.0, 0.0],
                           [0.0, 0.0, 0.0, 1.0]])
        self.Q = np.eye(4) * process_noise_std  # Process noise
        self.H = np.array([[1.0, 0.0, 0.0, 0.0],  # Measurement model
                           [0.0, 1.0, 0.0, 0.0]])
        self.R = np.eye(2) * measurement_noise_std  # Measurement noise
        self.last_updated = initial_detection.timestamp

    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.x, self.P

    def update(self, z):
        z = np.array([[float(z.x)], [float(z.y)]])
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x += K @ y
        self.P = (np.eye(4) - K @ self.H) @ self.P

    def get_position(self):
        return self.x[0, 0], self.x[1, 0]

def run_simulation(data, max_missed=3):
    tracks = {}
    current_time = data[0].timestamp

    for detection in data:
        # Update all tracks to current timestamp
        for track_id in list(tracks):
            while tracks[track_id].last_updated < detection.timestamp:
                tracks[track_id].predict()
                tracks[track_id].last_updated += 1

        # Data association (simple nearest neighbor for this example)
        if detection.id in tracks:
            tracks[detection.id].update(detection)
        else:
            # Initialize new track
            tracks[detection.id] = AdvancedKalmanFilter(detection)

        # Remove old tracks
        tracks = {tid: track for tid, track in tracks.items() if detection.timestamp - track.last_updated <= max_missed}

    # Final state of each track
    return {tid: (track.get_position(), track.last_updated) for tid, track in tracks.items()}

# Example data and class definitions for VehicleDetection
class VehicleDetection:
    def __init__(self, id, x, y, z, vx, vy, vz, timestamp):
        self.id = id
        self.x = x
        self.y = y
        self.z = z
        self.vx = vx
        self.vy = vy
        self.vz = vz
        self.timestamp = timestamp

# Simulated example data
example_data = [
    VehicleDetection(id="vehicle1", x=10, y=5, z=0, vx=1, vy=0.5, vz=0, timestamp=1),
    VehicleDetection(id="vehicle2", x=20, y=10, z=0, vx=2, vy=1, vz=0, timestamp=1),
    VehicleDetection(id="vehicle1", x=11, y=5.5, z=0, vx=1, vy=0.5, vz=0, timestamp=2),
    VehicleDetection(id="vehicle2", x=22, y=11, z=0, vx=2, vy=1, vz=0, timestamp=2),
    VehicleDetection(id="vehicle3", x=30, y=15, z=0, vx=3, vy=1.5, vz=0, timestamp=2)
]

# Run the simulation with the example data
final_track_states = run_simulation(example_data)
print(final_track_states)
